<a href="https://colab.research.google.com/github/blooming-daisy/Self_projects/blob/master/%5BGA4%5D_API_codes_(%EA%B3%B5%EC%9C%A0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. References

[1. GA4 Data API Guides](https://developers.google.com/analytics/devguides/reporting/data/v1/quickstart-client-libraries)

[2. Google API Code Samples](https://github.com/googleapis/python-analytics-data)

- GCP에서 API 서비스 계정 만든 후, 서비스 계정을 GA4에 추가해야 활용 가능;  또한 서비스 키를 다운받아서, 런타임 실행 전에 Colab에 업로드 해주기!


- Filter 관련 참고사항
    - To express dimension or metric filters. The fields in the same FilterExpression need to be **either all dimensions or all metrics**.

  - This message has oneof_ fields **(mutually exclusive fields)**. For each oneof, at most one member field can be set at the same time. Setting any member of the oneof automatically clears all other members.

  - Attributes:

        * and_group (google.analytics.data_v1beta.types.FilterExpressionList):
        The FilterExpressions in and_group have an AND relationship.

        * or_group (google.analytics.data_v1beta.types.FilterExpressionList):
        The FilterExpressions in or_group have an OR relationship.

        * not_expression (google.analytics.data_v1beta.types.FilterExpression):
        The FilterExpression is NOT of not_expression.

        * filter (google.analytics.data_v1beta.types.Filter):
        A primitive filter. In the same FilterExpression, all of the filter's field names need to be either all dimensions or all metrics.

In [ ]:
!pip install google-analytics-data

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.0/127.0 kB 2.4 MB/s eta 0:00:00


In [ ]:
import os
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import *
import pandas as pd

In [ ]:
# gapandas4 code 일부 사용

def get_headers(response):
  headers = []

  for header in response.dimension_headers:
    headers.append(header.name)

  for header in response.metric_headers:
    headers.append(header.name)

  return headers


def get_rows(response):
  rows = []
  for _row in response.rows:
    row = []
    for dimension in _row.dimension_values:
      row.append(dimension.value)
    for metric in _row.metric_values:
       row.append(metric.value)
       rows.append(row)
  return rows


def to_dataframe(response):
  headers = get_headers(response)
  rows = get_rows(response)
  df = pd.DataFrame(rows, columns=headers)
  return df


In [ ]:
service_account = 'fair-torus-367408-d1f4ff21fe88.json'

p_prop = '316465768'
t_prop = '345204778'

open(service_account)
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_account
client = BetaAnalyticsDataClient()

## 1. Privia 모니터링

In [ ]:
# Privia 모니터링 - 항공메인 함수

def privia_monitoring_air(startdate, enddate):
  # Using a default constructor instructs the client to use the credentials
  # specified in GOOGLE_APPLICATION_CREDENTIALS environment variable.

  request = RunReportRequest(
      property="properties/"+str(p_prop),
      dimensions=[Dimension(name='Date'), Dimension(name='customEvent:dimension10')],
      metrics=[Metric(name="screenPageViews"), Metric(name="totalUsers")],
      date_ranges=[DateRange(start_date=startdate, end_date=enddate)],
      order_bys = [OrderBy(dimension = OrderBy.DimensionOrderBy(dimension_name = "Date"), desc = False)],

      # dimension에 필터 설정하기
      dimension_filter = FilterExpression(
          and_group = FilterExpressionList(
              expressions =

                # filter: dimension10
                [FilterExpression(filter = Filter(
                      field_name = 'customEvent:dimension10',
                      string_filter = Filter.StringFilter(
                          match_type = Filter.StringFilter.MatchType.CONTAINS,
                          value = "air"))),

                 #filter: deviceCategory
                  FilterExpression(filter = Filter(
                      field_name = 'deviceCategory',
                      string_filter = Filter.StringFilter(value = 'desktop'))),


                #filter: pageType
                  FilterExpression(filter = Filter(
                      field_name = 'customEvent:pageType',
                      string_filter = Filter.StringFilter(
                          match_type = Filter.StringFilter.MatchType.CONTAINS,
                          value = "main")))],

              )
          )
      )


  response = client.run_report(request)

  df = to_dataframe(response).drop_duplicates()
  new_df = df.rename(columns = {'customEvent:dimension10' : '사업', 'screenPageViews' : 'PV', 'totalUsers' : "Users"})
  new_df = new_df.set_index('Date')

  return new_df



# Privia 모니터링 - 여행전체 함수

def privia_monitoring_ttl(startdate, enddate):
  # Using a default constructor instructs the client to use the credentials
  # specified in GOOGLE_APPLICATION_CREDENTIALS environment variable.

  request = RunReportRequest(
      property="properties/"+str(p_prop),
      dimensions=[Dimension(name='Date')],
      metrics=[Metric(name="screenPageViews"), Metric(name="totalUsers")],
      date_ranges=[DateRange(start_date=startdate, end_date=enddate)],
      order_bys = [OrderBy(dimension = OrderBy.DimensionOrderBy(dimension_name = "Date"), desc = False)],

      # dimension에 필터 설정하기
      dimension_filter = FilterExpression(
          and_group = FilterExpressionList(
              expressions =

                [ # filter: dimension10
                  FilterExpression(filter = Filter(
                      field_name = 'customEvent:dimension10',
                      string_filter = Filter.StringFilter(
                          match_type = Filter.StringFilter.MatchType.FULL_REGEXP,
                          value = "(air|htl|tna|comm|airms|airmb|pkg)"))),

                   #filter: dimension11
                  FilterExpression(filter = Filter(
                      field_name = 'customEvent:dimension11',
                      string_filter = Filter.StringFilter(value = 'pcWeb'))),

                 # filter: host name
                  FilterExpression(filter = Filter(
                      field_name = 'hostName',
                      string_filter = Filter.StringFilter(
                          match_type = Filter.StringFilter.MatchType.FULL_REGEXP,
                          value = "^(?im).+")))]
          )
       )
  )

  response = client.run_report(request)

  df = to_dataframe(response).drop_duplicates()
  new_df = df.rename(columns = {'customEvent:dimension10' : '사업', 'screenPageViews' : 'PV', 'totalUsers' : "Users"})
  new_df = new_df.set_index('Date')

  return new_df

In [ ]:
privia_monitoring_air('2023-07-01','2023-07-31').head()

,사업,PV,Users
Date,,,
20230701,air,500,273
20230702,air,603,320
20230703,air,1159,653
20230704,air,1241,664
20230705,air,1184,647


In [ ]:
privia_monitoring_ttl('2023-07-01', '2023-07-31').head()

,PV,Users
Date,,
20230701,23582,2632
20230702,27873,3086
20230703,54198,6254
20230704,55811,6203
20230705,53149,5881
